# <center> Chat Bot - Training Deep Learning Model!

In [57]:
#Importing all the Libraries
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
import random

import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

In [58]:
lemmatizer = WordNetLemmatizer()
word_file = open('C:/Users/mamoo/Desktop/Python_Coding/ChatBot/intents.json').read()
intents = json.loads(word_file)
#The following need to be downloaded the first time only!
# nltk.download('punkt')
# nltk.download('wordnet')

In [10]:
words = []
classes =[]
documents = []
ignore_list = ['.' , ',', '?', '!' ] # We will not consider these for our word list!
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #Tokenization
        word = nltk.word_tokenize(pattern)
        #Tokenization is just splitting the sentence/phrase into individual words
        
        words.extend(word)
        print(word, intent['tag'])
        #Add Documents in the Corpus
        documents.append((word, intent['tag']))
        #print(documents)
        #Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
#print(documents)

['Hi', 'there'] greeting
['How', 'are', 'you'] greeting
['Is', 'anyone', 'there', '?'] greeting
['Hey'] greeting
['Hola'] greeting
['Hello'] greeting
['Good', 'day'] greeting
['Bye'] goodbye
['See', 'you', 'later'] goodbye
['Goodbye'] goodbye
['Nice', 'chatting', 'to', 'you', ',', 'bye'] goodbye
['Till', 'next', 'time'] goodbye
['Thanks'] thanks
['Thank', 'you'] thanks
['That', "'s", 'helpful'] thanks
['Awesome', ',', 'thanks'] thanks
['Thanks', 'for', 'helping', 'me'] thanks
['How', 'you', 'could', 'help', 'me', '?'] options
['What', 'you', 'can', 'do', '?'] options
['What', 'help', 'you', 'provide', '?'] options
['How', 'you', 'can', 'be', 'helpful', '?'] options
['What', 'support', 'is', 'offered'] options
['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'] adverse_drug
['Open', 'adverse', 'drugs', 'module'] adverse_drug
['Give', 'me', 'a', 'list', 'of', 'drugs', 'causing', 'adverse', 'behavior'] adverse_drug
['List', 'all', 'drugs', 'suitable', 'for', 'patient', 'with', 'adv

In [14]:
#Lemmatization (This will give us the Roots for all the words)
words = [lemmatizer.lemmatize(w.lower())for w in words if w not in ignore_list]
words = sorted(list(set(words)))
#print(words)

In [18]:
# Sorting Classes
classes = sorted(list(set(classes)))
print(len(documents), 'Documents')
print(len(classes), 'Classes/Intents: ', classes)
print(len(words), 'Unique words (Post-Lemmatization)', words)

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb')) 


47 Documents
9 Classes/Intents:  ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
87 Unique words (Post-Lemmatization) ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [22]:
# Training & Testing Data
# Now we will convert the word list into Training Data!

training = []
output = [0]* len(classes)

for doc in documents:
    #Initializing Bag of Words
    bag = []
    
    #List of tokenized words for the pattern
    word_pattern = doc[0]
    
    #Lemmatizing each word 
    word_pattern = [lemmatizer.lemmatize(word.lower()) for word in word_pattern]
    
    #Create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_pattern else bag.append(0)
    
    # One Hot Encoding
    output_row = list(output)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

#Shuffling the features
random.shuffle(training)
training = np.array(training, dtype=object)

#Create training list. Train_x contains patterns, train_y contains Intents (That we are aiming to predict with our model)

train_x = list(training[:, 0])
train_y = list(training[:, 1])


In [52]:
# Model Training 
# Constructing the Deep Neural Network Architecture

model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]), ), activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

#Compiling the model
sgd = SGD(lr=0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer =sgd, metrics = ['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 128)               11264     
_________________________________________________________________
dropout_28 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_38 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_29 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_39 (Dense)             (None, 64)                4160      
_________________________________________________________________
dropout_30 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_40 (Dense)             (None, 9)               

In [53]:
#We'll use the Keras Early Stopping, based on Validation Accuracy, and for the Train & Validation set split, we'll use the inbuilt feature of the fit method! 
early_stopper = EarlyStopping(min_delta = 0.05 , patience = 15)

train_hist = model.fit(np.array(train_x), np.array(train_y),validation_split=0.3, epochs = 200, batch_size = 5, verbose = 1, callbacks = [early_stopper])


Train on 32 samples, validate on 15 samples
Epoch 1/200
32/32 [==============================] - 0s 8ms/step - loss: 2.2654 - accuracy: 0.0625 - val_loss: 2.1969 - val_accuracy: 0.2000
Epoch 2/200
32/32 [==============================] - 0s 2ms/step - loss: 2.1797 - accuracy: 0.1250 - val_loss: 2.2126 - val_accuracy: 0.0667
Epoch 3/200
32/32 [==============================] - 0s 2ms/step - loss: 2.1867 - accuracy: 0.1250 - val_loss: 2.2227 - val_accuracy: 0.0667
Epoch 4/200
32/32 [==============================] - 0s 1ms/step - loss: 2.1009 - accuracy: 0.2188 - val_loss: 2.2249 - val_accuracy: 0.1333
Epoch 5/200
32/32 [==============================] - 0s 1ms/step - loss: 2.0373 - accuracy: 0.3438 - val_loss: 2.2194 - val_accuracy: 0.1333
Epoch 6/200
32/32 [==============================] - 0s 2ms/step - loss: 2.0133 - accuracy: 0.2812 - val_loss: 2.1950 - val_accuracy: 0.1333
Epoch 7/200
32/32 [==============================] - 0s 2ms/step - loss: 1.9192 - accuracy: 0.2812 - val_loss:

In [88]:
# The validation accuracy isn't very good, but that's primarily becuase we are dealing with a very small dataset. 
# The purpose of this project was to understand how to handle text data and I'll be publishing more projects in the future to build models with large datasets, 
# since a pre-condition of working with Deep Learning Models is to have large amount of Data. 


In [55]:
# Now, we'll save our trained model to be used in for the Chatbot!

model.save('chat_bot_tr_model.h5', train_hist)